##### Copyright 2018 The TensorFlow Authors.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Automatically rewrite TF 1.x and compat.v1 API symbols

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/migrate/upgrade">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/migrate/upgrade.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/migrate/upgrade.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/migrate/upgrade.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />
    Download notebook</a>
  </td>
</table>


TensorFlow 2.x includes many API changes from TF 1.x and the `tf.compat.v1` APIs, such as reordering arguments, renaming symbols, and changing default values for parameters. Manually performing all of these modifications would be tedious and prone to error. To streamline the changes, and to make your transition to TF 2.x as seamless as possible, the TensorFlow team has created the `tf_upgrade_v2` utility to help transition legacy code to the new API.

Note: `tf_upgrade_v2` is installed automatically for TensorFlow 1.13 and later (including all TF 2.x builds).

Typical usage is like this:

<pre class="devsite-terminal devsite-click-to-copy prettyprint lang-bsh">
tf_upgrade_v2 \
  --intree my_project/ \
  --outtree my_project_v2/ \
  --reportfile report.txt
</pre>

It will accelerate your upgrade process by converting existing TensorFlow 1.x Python scripts to TensorFlow 2.x.

The conversion script automates many mechanical API transformations, though many APIs cannot be automatically migrated. It is also not able to fully make your code compatible with TF2 behaviors and APIs. So, it is only a part of your migration journey.

## Compatibility modules

Certain API symbols can not be upgraded simply by using a string replacement. Those that cannot be automatically upgraded will be mapped to their locations in the `compat.v1` module. This module replaces TF 1.x symbols like `tf.foo` with the equivalent `tf.compat.v1.foo` reference. If you are already using `compat.v1` APIs by importing TF via `import tensorflow.compat.v1 as tf`, the `tf_upgrade_v2` script will attempt to convert these usages to the non-compat APIs where possible. Note that while some `compat.v1` APIs are compatible with TF2.x behaviors, many are not. Therefore, it's recommended to manually proofread replacements and migrate them to new APIs in the `tf.*` namespace instead of `tf.compat.v1` namespace as quickly as possible.

Because of TensorFlow 2.x module deprecations (for example, `tf.flags` and `tf.contrib`), some changes can not be worked around by switching to `compat.v1`. Upgrading this code may require using an additional library (for example, [`absl.flags`](https://github.com/abseil/abseil-py)) or switching to a package in [tensorflow/addons](http://www.github.com/tensorflow/addons).


## Recommended upgrade process

The rest of this guide demonstrates how to use the symbol-rewriting script. While the script is easy to use, it is strongly recommended that you use the script as part of the following process: 

1. **Unit Test**: Ensure that the code you’re upgrading has a unit test suite with reasonable coverage. This is Python code, so the language won’t protect you from many classes of mistakes. Also ensure that any dependency you have has already been upgraded to be compatible with TensorFlow 2.x.

1. **Install TensorFlow 1.15**: Upgrade your TensorFlow to the latest TensorFlow 1.x version, at least 1.15. This includes the final TensorFlow 2.0 API in `tf.compat.v2`.

1. **Test With 1.15**: Ensure your unit tests pass at this point. You’ll be running them repeatedly as you upgrade so starting from green is important.

1. **Run the upgrade script**: Run `tf_upgrade_v2` on your entire source tree, tests included. This will upgrade your code to a format where it only uses symbols available in TensorFlow 2.0. Deprecated symbols will be accessed with `tf.compat.v1`. These will eventually require manual attention, but not immediately.

1. **Run the converted tests with TensorFlow 1.15**: Your code should still run fine in TensorFlow 1.15. Run your unit tests again. Any error in your tests here means there’s a bug in the upgrade script. [Please let us know](https://github.com/tensorflow/tensorflow/issues).

1. **Check the upgrade report for warnings and errors**: The script writes a report file that explains any conversions you should double-check, or any manual action you need to take. For example: Any remaining instances of contrib will require manual action to remove. Please consult [the RFC for more instructions](https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md). 

1. **Install TensorFlow 2.x**: At this point it should be safe to switch to TensorFlow 2.x binaries, even if you are running with legacy behaviors

1. **Test with `v1.disable_v2_behavior`**: Re-running your tests with a `v1.disable_v2_behavior()` in the tests' main function should give the same results as running under 1.15.

1. **Enable V2 Behavior**: Now that your tests work using the TF2 binaries, you can now begin migrating your code to avoiding `tf.estimator`s and only using supported TF2 behaviors (with no TF2 behavior disabling). See the [Migration guides](https://tensorflow.org/guide/migrate) for details.

## Using the symbol-rewriting `tf_upgrade_v2` script


### Setup

Before getting started ensure that TensorFlow 2.x is installed.

In [2]:
import tensorflow as tf

print(tf.__version__)

2.11.0


Clone the [tensorflow/models](https://github.com/tensorflow/models) git repository so you have some code to test on:

In [ ]:
!git clone --branch r1.13.0 --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...


remote: Enumerating objects: 2927, done.
remote: Counting objects:   0% (1/2927)
remote: Counting objects:   1% (30/2927)
remote: Counting objects:   2% (59/2927)
remote: Counting objects:   3% (88/2927)
remote: Counting objects:   4% (118/2927)
remote: Counting objects:   5% (147/2927)
remote: Counting objects:   6% (176/2927)


remote: Counting objects:   8% (235/2927)
remote: Counting objects:   9% (264/2927)
remote: Counting objects:  10% (293/2927)
remote: Counting objects:  11% (322/2927)
remote: Counting objects:  12% (352/2927)
remote: Counting objects:  13% (381/2927)
remote: Counting objects:  14% (410/2927)
remote: Counting objects:  15% (440/2927)
remote: Counting objects:  16% (469/2927)
remote: Counting objects:  17% (498/2927)
remote: Counting objects:  18% (527/2927)
remote: Counting objects:  19% (557/2927)
remote: Counting objects:  20% (586/2927)
remote: Counting objects:  21% (615/2927)
remote: Counting objects:  22% (644/2927)
remote: Counting objects:  23% (674/2927)
remote: Counting objects:  24% (703/2927)
remote: Counting objects:  25% (732/2927)
remote: Counting objects:  26% (762/2927)
remote: Counting objects:  27% (791/2927)
remote: Counting objects:  28% (820/2927)
remote: Counting objects:  29% (849/2927)
remote: Counting objects:  30% (879/2927)
remote: Counting objects:  31% (90

remote: Compressing objects:   7% (170/2428)
remote: Compressing objects:   8% (195/2428)
remote: Compressing objects:   9% (219/2428)
remote: Compressing objects:  10% (243/2428)
remote: Compressing objects:  11% (268/2428)


remote: Compressing objects:  13% (316/2428)
remote: Compressing objects:  14% (340/2428)
remote: Compressing objects:  15% (365/2428)
remote: Compressing objects:  16% (389/2428)
remote: Compressing objects:  17% (413/2428)


...

Updating files:  79% (2187/2768)
Updating files:  80% (2215/2768)


Updating files:  81% (2248/2768)
Updating files:  82% (2270/2768)


Updating files:  85% (2353/2768)
Updating files:  86% (2381/2768)
Updating files:  87% (2409/2768)


Updating files:  89% (2464/2768)
Updating files:  90% (2492/2768)
Updating files:  91% (2519/2768)
Updating files:  92% (2547/2768)


Updating files:  94% (2602/2768)
Updating files:  95% (2630/2768)
Updating files:  96% (2658/2768)
Updating files:  97% (2685/2768)
Updating files:  98% (2713/2768)
Updating files:  99% (2741/2768)
Updating files: 100% (2768/2768)
Updating files: 100% (2768/2768), done.


### Read the help

The script should be installed with TensorFlow. Here is the builtin help:

In [ ]:
!tf_upgrade_v2 -h

### Example TF1 code

Here is a simple TensorFlow 1.0 script:

In [ ]:
!head -n 65 models/samples/cookbook/regression/custom_regression.py | tail -n 10

  # Calculate loss using mean squared error
  average_loss = tf.losses.mean_squared_error(labels, predictions)

  # Pre-made estimators use the total_loss instead of the average,
  # so report total_loss for compatibility.
  batch_size = tf.shape(labels)[0]
  total_loss = tf.to_float(batch_size) * average_loss

  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = params.get("optimizer", tf.train.AdamOptimizer)


With TensorFlow 2.x installed it does not run:

In [ ]:
!(cd models/samples/cookbook/regression && python custom_regression.py)

2022-12-14 03:12:40.239154: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-14 03:12:40.239244: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-14 03:12:40.239254: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


Traceback (most recent call last):
  File "/tmpfs/src/temp/site/en/guide/migrate/models/samples/cookbook/regression/custom_regression.py", line 162, in <module>
    tf.logging.set_verbosity(tf.logging.INFO)
AttributeError: module 'tensorflow' has no attribute 'logging'


### Single file

The script can be run on a single Python file:

In [7]:
%cd /content/drive/MyDrive/
!ls


/content/drive/MyDrive
 01_ojt_0830_822.gsheet
 01_ojt_0831_536.gsheet
 01_ojt_0906_825.gsheet
 01_ojt_0908_826.gsheet
 01_ojt_0909.gsheet
 01_ojt_1025.gsheet
 01_ojt_1101.gsheet
'10 11.gslides'
'10 21.gslides'
'10 27.gslides'
'10 7.gslides'
 1110
'11 10.gslides'
'11 17.gslides'
'11 22.gslides'
'11 24.gslides'
'11 5.gslides'
'12 15.gslides'
'12 1.gslides'
'12 8.gslides'
 2020情報ネットワーク
'2021年第１回実務訓練履修説明会レポート (1).gdoc'
 2021年第１回実務訓練履修説明会レポート.gdoc
 2022_派_様式H-1.xlsx
'203344report10 (1).pdf'
 203344report10.pdf
 203344report11.pdf
 203344report12.pdf
 203344report13.pdf
 203344report14.pdf
'203344report (1).zip'
 203344_report3.zip
 203344report5.zip
 203344report.pdf
 203344report_repost.zip
 203344report.zip
 203344_takagi_interimposter.gslides
'203344_演習問題1 (1).pdf'
 203344_演習問題1.pdf
'203344離散数学論第１講レポート (1).pdf'
 203344離散数学論第１講レポート.pdf
 203344_高木勇弥.pdf
 203344_高木勇弥_実務訓練履修書.gdoc
 203344_高木勇弥_実務訓練最終報告書.gdoc
 2_motivation_1.gdoc
'7 12.gslides'
'7 19.gslides'
'7 21.gslides'
'7 28.gslides

In [ ]:
!tf_upgrade_v2 \
  --infile models/samples/cookbook/regression/custom_regression.py \
  --outfile /tmp/custom_regression_v2.py

2022-12-14 03:12:42.626769: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-14 03:12:42.626859: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-14 03:12:42.626869: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


INFO line 38:8: Renamed 'tf.feature_column.input_layer' to 'tf.compat.v1.feature_column.input_layer'
INFO line 43:10: Renamed 'tf.layers.dense' to 'tf.compat.v1.layers.dense'
INFO line 46:17: Renamed 'tf.layers.dense' to 'tf.compat.v1.layers.dense'
INFO line 57:17: tf.losses.mean_squared_error requires manual check. tf.losses have been replaced with object oriented versions in TF 2.0 and after. The loss function calls have been converted to compat.v1 for backward compatibility. Please update these calls to the TF 2.0 versions.
INFO line 57:17: Renamed 'tf.losses.mean_squared_error' to 'tf.compat.v1.losses.mean_squared_error'
INFO line 61:15: Added keywords to args of function 'tf.shape'
INFO line 62:15: Changed tf.to_float call to tf.cast(..., dtype=tf.float32).
INFO line 65:40: Renamed 'tf.train.AdamOptimizer' to 'tf.compat.v1.train.AdamOptimizer'
INFO line 68:39: Renamed 'tf.train.get_global_step' to 'tf.compat.v1.train.get_global_step'
INFO line 83:9: tf.metrics.root_mean_squared_er

The script will print errors if it can not find a fix for the code. 

### Directory tree

Typical projects, including this simple example, will use much more than one file. Typically want to update an entire package, so the script can also be run on a directory tree:

In [ ]:
%

In [ ]:
# update the .py files and copy all the other files to the outtree
!tf_upgrade_v2 \
    --intree models/samples/cookbook/regression/ \
    --outtree regression_v2/ \
    --reportfile tree_report.txt

2022-12-14 03:12:45.060845: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-14 03:12:45.060935: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-14 03:12:45.060946: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


INFO line 58:10: tf.estimator.LinearRegressor: Default value of loss_reduction has been changed to SUM_OVER_BATCH_SIZE; inserting old default value tf.keras.losses.Reduction.SUM.

INFO line 101:2: Renamed 'tf.logging.set_verbosity' to 'tf.compat.v1.logging.set_verbosity'
INFO line 101:27: Renamed 'tf.logging.INFO' to 'tf.compat.v1.logging.INFO'
INFO line 102:2: Renamed 'tf.app.run' to 'tf.compat.v1.app.run'
INFO line 38:8: Renamed 'tf.feature_column.input_layer' to 'tf.compat.v1.feature_column.input_layer'
INFO line 43:10: Renamed 'tf.layers.dense' to 'tf.compat.v1.layers.dense'
INFO line 46:17: Renamed 'tf.layers.dense' to 'tf.compat.v1.layers.dense'
INFO line 57:17: tf.losses.mean_squared_error requires manual check. tf.losses have been replaced with object oriented versions in TF 2.0 and after. The loss function calls have been converted to compat.v1 for backward compatibility. Please update these calls to the TF 2.0 versions.
INFO line 57:17: Renamed 'tf.losses.mean_squared_error' 

INFO line 162:2: Renamed 'tf.logging.set_verbosity' to 'tf.compat.v1.logging.set_verbosity'
INFO line 162:27: Renamed 'tf.logging.INFO' to 'tf.compat.v1.logging.INFO'
INFO line 163:2: Renamed 'tf.app.run' to 'tf.compat.v1.app.run'
INFO line 72:10: tf.estimator.DNNRegressor: Default value of loss_reduction has been changed to SUM_OVER_BATCH_SIZE; inserting old default value tf.keras.losses.Reduction.SUM.

INFO line 96:2: Renamed 'tf.logging.set_verbosity' to 'tf.compat.v1.logging.set_verbosity'
INFO line 96:27: Renamed 'tf.logging.INFO' to 'tf.compat.v1.logging.INFO'
INFO line 97:2: Renamed 'tf.app.run' to 'tf.compat.v1.app.run'
WARNING line 125:15: Changing dataset.make_one_shot_iterator() to tf.compat.v1.data.make_one_shot_iterator(dataset). Please check this transformation.



INFO line 40:7: Renamed 'tf.test.mock' to 'tf.compat.v1.test.mock'
INFO line 82:10: tf.estimator.LinearRegressor: Default value of loss_reduction has been changed to SUM_OVER_BATCH_SIZE; inserting old default value tf.keras.losses.Reduction.SUM.

INFO line 105:2: Renamed 'tf.logging.set_verbosity' to 'tf.compat.v1.logging.set_verbosity'
INFO line 105:27: Renamed 'tf.logging.INFO' to 'tf.compat.v1.logging.INFO'
INFO line 106:2: Renamed 'tf.app.run' to 'tf.compat.v1.app.run'
TensorFlow 2.0 Upgrade Script
-----------------------------
Converted 7 files
Detected 1 issues that require attention
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
File: models/samples/cookbook/regression/automobile_data.py
--------------------------------------------------------------------------------
models/samples/cookbook/regression/automobile_data.py:125:15: WARNING: Changing dataset.make_one_sho

Note the one warning about the `dataset.make_one_shot_iterator` function.

Now the script works in with TensorFlow 2.x:

Note that because the `tf.compat.v1` module is included in TF 1.15, the converted script will also run in TensorFlow 1.15.

In [ ]:
!(cd regression_v2 && python custom_regression.py 2>&1) | tail

I1214 03:12:56.679630 140626957535040 estimator.py:2083] Saving dict for global step 1000: global_step = 1000, loss = 844.2114, rmse = 4.1937733
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmpfs/tmp/tmp99nj9v2r/model.ckpt-1000
I1214 03:12:56.722749 140626957535040 estimator.py:2143] Saving 'checkpoint_path' summary for global step 1000: /tmpfs/tmp/tmp99nj9v2r/model.ckpt-1000
Tensor("IteratorGetNext:25", shape=(None,), dtype=float64, device=/device:CPU:0)
Tensor("Squeeze:0", shape=(None,), dtype=float32)

********************************************************************************

RMS error for the test set: $4194



## Detailed report

The script also reports a list of detailed changes. In this example it found one possibly unsafe transformation and included a warning at the top of the file: 

In [ ]:
!head -n 20 tree_report.txt

TensorFlow 2.0 Upgrade Script
-----------------------------
Converted 7 files
Detected 1 issues that require attention
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
File: models/samples/cookbook/regression/automobile_data.py
--------------------------------------------------------------------------------
models/samples/cookbook/regression/automobile_data.py:125:15: WARNING: Changing dataset.make_one_shot_iterator() to tf.compat.v1.data.make_one_shot_iterator(dataset). Please check this transformation.

Detailed log follows:

Input tree: 'models/samples/cookbook/regression/'
--------------------------------------------------------------------------------
Processing file 'models/samples/cookbook/regression/linear_regression.py'
 outputting to 'regression_v2/linear_regression.py'


Note again the one warning about the `Dataset.make_one_shot_iterator function`.

In other cases the output will explain the reasoning for non-trivial changes:

In [ ]:
%%writefile dropout.py
import tensorflow as tf

d = tf.nn.dropout(tf.range(10), 0.2)
z = tf.zeros_like(d, optimize=False)

Writing dropout.py


In [ ]:
!tf_upgrade_v2 \
  --infile dropout.py \
  --outfile dropout_v2.py \
  --reportfile dropout_report.txt > /dev/null

2022-12-14 03:12:59.002341: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-14 03:12:59.002433: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-14 03:12:59.002444: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [ ]:
!cat dropout_report.txt

TensorFlow 2.0 Upgrade Script
-----------------------------
Converted 1 files
Detected 0 issues that require attention
--------------------------------------------------------------------------------
Detailed log follows:

--------------------------------------------------------------------------------
Processing file 'dropout.py'
 outputting to 'dropout_v2.py'
--------------------------------------------------------------------------------

3:4: INFO: Changing keep_prob arg of tf.nn.dropout to rate, and recomputing value.

4:4: INFO: Renaming tf.zeros_like to tf.compat.v1.zeros_like because argument optimize is present. tf.zeros_like no longer takes an optimize argument, and behaves as if optimize=True. This call site specifies something other than optimize=True, so it was converted to compat.v1.
--------------------------------------------------------------------------------



Here is the modified file contents, note how the script adds argument names to deal with moved and renamed arguments:

In [ ]:
!cat dropout_v2.py

import tensorflow as tf

d = tf.nn.dropout(tf.range(10), rate=1 - (0.2))
z = tf.compat.v1.zeros_like(d, optimize=False)


A larger project might contain a few errors. For example convert the deeplab model:

In [ ]:
!tf_upgrade_v2 \
    --intree models/research/deeplab \
    --outtree deeplab_v2 \
    --reportfile deeplab_report.txt > /dev/null

2022-12-14 03:13:01.666677: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-14 03:13:01.666763: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-14 03:13:01.666774: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


It produced the output files:

In [ ]:
!ls deeplab_v2

README.md	datasets	    input_preprocess.py        train.py
__init__.py	deeplab_demo.ipynb  local_test.sh	       utils
common.py	eval.py		    local_test_mobilenetv2.sh  vis.py
common_test.py	export_model.py     model.py
core		g3doc		    model_test.py


But there were errors. The report will help you pin-point what you need to fix before this will run. Here are the first three errors:

In [ ]:
!cat deeplab_report.txt | grep -i models/research/deeplab | grep -i error | head -n 3

models/research/deeplab/model.py:60:7: ERROR: Using member tf.contrib.slim in deprecated module tf.contrib. tf.contrib.slim cannot be converted automatically. tf.contrib will not be distributed with TensorFlow 2.0, please consider an alternative in non-contrib TensorFlow, a community-maintained repository such as tensorflow/addons, or fork the required code.
models/research/deeplab/train.py:29:7: ERROR: Using member tf.contrib.slim in deprecated module tf.contrib. tf.contrib.slim cannot be converted automatically. tf.contrib will not be distributed with TensorFlow 2.0, please consider an alternative in non-contrib TensorFlow, a community-maintained repository such as tensorflow/addons, or fork the required code.
models/research/deeplab/eval.py:28:7: ERROR: Using member tf.contrib.slim in deprecated module tf.contrib. tf.contrib.slim cannot be converted automatically. tf.contrib will not be distributed with TensorFlow 2.0, please consider an alternative in non-contrib TensorFlow, a comm

## "Safety" mode

The conversion script also has a less invasive `SAFETY` mode that simply changes the imports to use the `tensorflow.compat.v1` module:

In [ ]:
!cat dropout.py

import tensorflow as tf

d = tf.nn.dropout(tf.range(10), 0.2)
z = tf.zeros_like(d, optimize=False)


In [ ]:
!tf_upgrade_v2 --mode SAFETY --infile dropout.py --outfile dropout_v2_safe.py > /dev/null

2022-12-14 03:13:05.895833: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-14 03:13:05.895924: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-14 03:13:05.895951: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [ ]:
!cat dropout_v2_safe.py

import tensorflow.compat.v1 as tf

d = tf.nn.dropout(tf.range(10), 0.2)
z = tf.zeros_like(d, optimize=False)


As you can see this doesn't upgrade your code, but does allow TensorFlow 1 code to run against TensorFlow 2 binaries. Note that this does not mean your code is running supported TF 2.x behaviors!

## Caveats

- Do not update parts of your code manually before running this script. In particular, functions that have had reordered arguments like `tf.math.argmax` or `tf.batch_to_space` cause the script to incorrectly add keyword arguments that mismap your existing code.

- The script assumes that `tensorflow` is imported using `import tensorflow as tf`, or `import tensorflow.compat.v1 as tf`.

- This script does not reorder arguments. Instead, the script adds keyword arguments to functions that have their arguments reordered.

- Check out [tf2up.ml](https://github.com/lc0/tf2up) for a convenient tool to upgrade Jupyter
  notebooks and Python files in a GitHub repository.

To report upgrade script bugs or make feature requests, please file an issue on [GitHub](https://github.com/tensorflow/tensorflow/issues).